In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

# 1. Load and Preprocess Data
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# 2. Define CNN Architecture
class CIFAR10_CNN(nn.Module):
    def __init__(self):
        super(CIFAR10_CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)
        
    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))  # 32x32 -> 16x16
        x = self.pool(nn.functional.relu(self.conv2(x)))  # 16x16 -> 8x8
        x = x.view(-1, 64 * 8 * 8)
        x = self.dropout(x)
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = CIFAR10_CNN()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# 3. Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)

# 4. Training Loop
def train(epoch):
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(trainloader, desc=f'Epoch {epoch+1}'):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    train_loss = running_loss / len(trainloader)
    return train_loss

# 5. Evaluation Function
def evaluate():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# 6. Main Training Process
for epoch in range(25):
    train_loss = train(epoch)
    val_accuracy = evaluate()
    scheduler.step(train_loss)
    
    print(f'Epoch {epoch+1}: Loss={train_loss:.4f}, Accuracy={val_accuracy:.2f}%')

# 7. Final Evaluation
final_accuracy = evaluate()
print(f'Final Test Accuracy: {final_accuracy:.2f}%')

Epoch 1: 100%|██████████| 391/391 [00:18<00:00, 20.66it/s]


Epoch 1: Loss=1.5801, Accuracy=51.33%


Epoch 2: 100%|██████████| 391/391 [00:23<00:00, 16.45it/s]


Epoch 2: Loss=1.2562, Accuracy=60.51%


Epoch 3: 100%|██████████| 391/391 [00:16<00:00, 23.32it/s]


...

Epoch 23: Loss=0.6838, Accuracy=77.05%


Epoch 24: 100%|██████████| 391/391 [00:27<00:00, 14.46it/s]


Epoch 24: Loss=0.6791, Accuracy=76.64%


Epoch 25: 100%|██████████| 391/391 [00:26<00:00, 14.89it/s]


Epoch 25: Loss=0.6733, Accuracy=76.66%
Final Test Accuracy: 76.44%
